In [ ]:
! pip install KDE-diffusion

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

# Define the file paths
file_path1 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_training_data.csv'
file_path2 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_testing_data.csv'
file_path3 = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_training_data.csv'
file_path4  = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_testing_data.csv'
# Use pandas to read the CSV files and then convert them to NumPy arrays
caltech_train = pd.read_csv(file_path1).values
caltech_test = pd.read_csv(file_path2).values

JPL_train = pd.read_csv(file_path3).values
JPL_test=pd.read_csv(file_path4).values

#Data Processing for DKDE

In [ ]:
#Remove row number (in 1st column)
caltech_train=caltech_train[:,1:]
caltech_test=caltech_test[:,1:]

JPL_train=JPL_train[:,1:]
JPL_test=JPL_test[:,1:]

In [ ]:
#Remove departure time (2nd column)
# Convert arrival date to hour and find day of the week
from datetime import datetime

def convert_time_and_day(data_array):
    """
    Converts the time from HH:MM to HH.XX format and appends the day of the week to it.
    Also, removes the second column.
    """
    transformed_data = []
    for row in data_array:
        # Convert the arrival time to HH.XX format
        time_obj = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
        new_time = time_obj.hour + (time_obj.minute / 60.0)

        # Convert the date to a day of the week
        day_of_week = time_obj.strftime('%A')
        new_time = str(new_time) + " " + day_of_week

        # Create a new row excluding the second column
        new_row = [new_time] + list(row[2:])
        transformed_data.append(new_row)

    return np.array(transformed_data)

caltech_train=convert_time_and_day(caltech_train)
caltech_test=convert_time_and_day(caltech_test)
JPL_train=convert_time_and_day(JPL_train)
JPL_test=convert_time_and_day(JPL_test)

In [ ]:
def day_to_number(day):
    """Converts a day of the week to its corresponding discrete value."""
    days = {
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7
    }
    return days[day]

def separate_time_and_day(data_array):
    """
    Separates the time and day in the given column,
    and converts the day into a discrete value between 1 and 7.

    """
    transformed_data = []
    for row in data_array:
        time_day_str = row[0]
        time, day = time_day_str.split()
        time = float(time)
        day_num = day_to_number(day)

        # Create a new row with separated time and day number
        new_row = [time, day_num] + list(row[1:])
        transformed_data.append(new_row)

    return np.array(transformed_data)

caltech_train=separate_time_and_day(caltech_train)
caltech_test=separate_time_and_day(caltech_test)
JPL_train=separate_time_and_day(JPL_train)
JPL_test=separate_time_and_day(JPL_test)

In [ ]:
#Make training and testing set have the same user IDs
users_from_training_caltech = set(caltech_train[:, 3])
mask_caltech = np.isin(caltech_test[:, 3], list(users_from_training_caltech))
caltech_test = caltech_test[mask_caltech]
users_from_testing_caltech = set(caltech_test[:, 3])
mask_caltech = np.isin(caltech_train[:, 3], list(users_from_testing_caltech))
caltech_train = caltech_train[mask_caltech]

users_from_training = set(JPL_train[:, 3])
mask = np.isin(JPL_test[:, 3], list(users_from_training))
JPL_test = JPL_test[mask]
users_from_testing = set(JPL_test[:, 3])
mask = np.isin(JPL_train[:, 3], list(users_from_testing))
JPL_train = JPL_train[mask]

In [ ]:
caltech_train = np.array(caltech_train, dtype='float')
caltech_test = np.array(caltech_test, dtype='float')
JPL_train = np.array(JPL_train, dtype='float')
JPL_test = np.array(JPL_test, dtype='float')

#Construct DKDE of stay duration and energy consumption

In [ ]:
from sklearn.preprocessing import StandardScaler
from kde_diffusion import kde2d
def dkde(train_data,test_data,user_id):
    # Filter training and testing data for the specific user
    user_train_data = train_data[train_data[:, 3] == user_id]
    user_test_data = test_data[test_data[:, 3] == user_id]

    # Extract columns for stay duration and energy consumption for both train and test sets
    X_train = user_train_data[:, 4]  # Stay duration
    Y_train = user_train_data[:, 2] # Energy consumption

    X_test = user_test_data[:, 4]
    y_test = user_test_data[:, 2]


    (density, grid, bandwidth) = kde2d(X_train, Y_train, n=128, limits=None)

    def predict_y_given_x(new_x, grid, density):
        # Find the closest x index
        x_idx = np.argmin(np.abs(grid[0] - new_x))

        # Get the y values and their corresponding densities for the given x
        y_values = grid[1]
        y_densities = density[x_idx]

        # Find the y with the maximum density
        predicted_y = y_values[np.argmax(y_densities)]

        return predicted_y

    # Predict y values for X_test using the density estimate
    y_pred = [predict_y_given_x(x_val, grid, density) for x_val in X_test]

    # Calculate user SMAPE
    n = len(y_test)
    smape_val = (1/ n) * np.sum(np.abs(y_test - y_pred) / (np.abs(y_test+y_pred)))*100

    return smape_val,density,grid



In [ ]:
# Test the function
user_ids_caltech = np.unique(np.concatenate((caltech_train[:, 3], caltech_test[:, 3])))
user_ids_JPL = np.unique(np.concatenate((JPL_train[:, 3], JPL_test[:, 3])))

smape_list_caltech=[]
smape_list_JPL=[]

for user_id in user_ids_caltech:
    smape, density, grid= dkde(caltech_train, caltech_test, user_id)
    smape_list_caltech.append(smape)
for user_id in user_ids_JPL:
    smape,density,grid = dkde(JPL_train, JPL_test, user_id)
    smape_list_JPL.append(smape)

In [ ]:
#Calculate average SMAPE for each location
no_caltech_users=len(user_ids_caltech)
caltech_smape=sum(smape_list_caltech)/no_caltech_users

no_JPL_users=len(user_ids_JPL)
JPL_smape=sum(smape_list_JPL)/no_JPL_users

print(f"Caltech SMAPE: {caltech_smape}")
print(f"JPL SMAPE: {JPL_smape}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 3D Plotting code
X, Y = np.meshgrid(grid[0], grid[1])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(Y,X, density, cmap='viridis')
ax.set_xlabel('Energy Consumption')
ax.set_ylabel('Stay Duration')
ax.set_zlabel('Density')
fig.colorbar(surf)
plt.show()

#Calculate SMAPE for different thresholds

In [ ]:
# Initialize dictionaries to store user-wise SMAPE values
user_smape_JPL = {}

# Loop through each user ID in the JPL dataset
for user_id in user_ids_JPL:
    smape, density, grid = dkde(JPL_train, JPL_test, user_id)
    user_smape_JPL[user_id] = smape  # Store the SMAPE in the dictionary

average_smape_JPL = np.mean(list(user_smape_JPL.values()))

print(f"Average SMAPE for JPL dataset: {average_smape_JPL:.2f}%")

In [ ]:
R_DE={'169': 2.119068507176728,
 '171': 3.7115405017002776,
 '176': 2.9977726188370353,
 '220': 4.170165871125364,
 '322': 2.974319275358279,
 '334': 1.54796573662954,
 '335': 3.9915670302142447,
 '346': 4.687498008228514,
 '365': 4.400963644915041,
 '368': 3.4312414172978474,
 '372': 2.9575939245001077,
 '374': 3.839538311235034,
 '378': 3.1515844041039993,
 '382': 4.148262279562261,
 '404': 3.097855936467447,
 '405': 4.732202935285684,
 '406': 1.158211936837657,
 '409': 2.886901319079918,
 '410': 4.280915725084575,
 '416': 4.148262279562261,
 '436': 3.3808089672729613,
 '444': 3.0622185077071102,
 '458': 4.148262279562261,
 '467': 3.777008295296614,
 '474': 2.6165283654578775,
 '476': 3.7356597962280196,
 '481': 4.045488808054726,
 '483': 2.755038189921663,
 '507': 3.7804325624313817,
 '526': 2.4478241428939165,
 '531': 3.5032226226713927,
 '537': 3.1955976636365735,
 '551': 4.767567980735159,
 '553': 2.555583224030839,
 '576': 3.380624050295972,
 '577': 4.031441936975631,
 '581': 2.9304059447845647,
 '592': 3.056683078202939,
 '607': 4.625780698041485,
 '651': 3.492080210083478,
 '726': 3.2841952906487855,
 '742': 3.9023067062499264,
 '826': 3.375181253576597,
 '933': 3.9394875755880188}

R_DE = {int(key): value for key, value in R_DE.items()}

In [ ]:
import numpy as np

# Initialize dictionaries to store user-wise SMAPE values
user_smape_JPL_less_than_3_5 = {}
user_smape_JPL_greater_than_3_5 = {}

# Loop through each user ID in the JPL dataset
for user_id in user_ids_JPL:
    smape, density, grid = dkde(JPL_train, JPL_test, user_id)

    if R_DE.get(user_id, 0) < 3.5:
        user_smape_JPL_less_than_3_5[user_id] = smape
    else:
        user_smape_JPL_greater_than_3_5[user_id] = smape
        print(f'Above 3.5: User {user_id} SMAPE: {smape:.2f}%')

# Calculate the average SMAPE for users with ratio < 4
average_smape_JPL_less_than_3_5 = np.mean(list(user_smape_JPL_less_than_3_5.values())) if user_smape_JPL_less_than_3_5 else 0

# Calculate the average SMAPE for users with ratio > 4
average_smape_JPL_greater_than_3_5 = np.mean(list(user_smape_JPL_greater_than_3_5.values())) if user_smape_JPL_greater_than_3_5 else 0

print(f"Average SMAPE for JPL dataset (Ratio < 3.5): {average_smape_JPL_less_than_3_5:.2f}%")
print(f"Average SMAPE for JPL dataset (Ratio > 3.5): {average_smape_JPL_greater_than_3_5:.2f}%")


In [ ]:
import numpy as np

# Initialize dictionaries to store user-wise SMAPE values
user_smape_JPL_less_than_4 = {}
user_smape_JPL_greater_than_4 = {}

# Loop through each user ID in the JPL dataset
for user_id in user_ids_JPL:
    smape, density, grid = dkde(JPL_train, JPL_test, user_id)

    if R_DE.get(user_id, 0) < 4:
        user_smape_JPL_less_than_4[user_id] = smape
    else:
        user_smape_JPL_greater_than_4[user_id] = smape
        print(f'Above 4: User {user_id} SMAPE: {smape:.2f}%')

# Calculate the average SMAPE for users with ratio < 4
average_smape_JPL_less_than_4 = np.mean(list(user_smape_JPL_less_than_4.values())) if user_smape_JPL_less_than_4 else 0

# Calculate the average SMAPE for users with ratio > 4
average_smape_JPL_greater_than_4 = np.mean(list(user_smape_JPL_greater_than_4.values())) if user_smape_JPL_greater_than_4 else 0

print(f"Average SMAPE for JPL dataset (Ratio < 4): {average_smape_JPL_less_than_4:.2f}%")
print(f"Average SMAPE for JPL dataset (Ratio > 4): {average_smape_JPL_greater_than_4:.2f}%")
